In [108]:
from src import Trainable
from src.agents import PPO
import torch
from src.utils import get_device

In [109]:
agent: PPO = Trainable.load_checkpoint('checkpoints\\breakout-ppo-1744484349\\9765.pt')

c:\Users\Ben Rall\AppData\Local\Programs\Python\Python312\Lib\site-packages\gymnasium\wrappers\rendering.py:283: UserWarning: WARN: Overwriting existing videos at d:\Coding\Year 3 Project\logs folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [110]:
print(agent.actor.model[-1].out_features)

4


In [111]:
obs = torch.Tensor(agent.envs.reset(seed=1)[0])

In [112]:
print(obs)

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          

In [113]:
print(obs.shape)
print(agent.actor(obs))

torch.Size([8, 4, 84, 84])


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [ ]:
print(agent)

PPO(
  (actor): PPOAtariActor(
    (model): Sequential(
      (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
      (3): ReLU()
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
      (7): Linear(in_features=3136, out_features=512, bias=True)
      (8): ReLU()
      (9): Linear(in_features=512, out_features=4, bias=True)
    )
  )
  (critic): PPOAtariCritic(
    (model): Sequential(
      (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
      (3): ReLU()
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
      (7): Linear(in_features=3136, out_features=512, bias=True)
      (8): ReLU()
      (9): Linear(in_features=512, out_features=1, bias=True)
    )
  )
)


In [ ]:
from torch import nn
from src.models import Model

In [ ]:
def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(),
    )

class Shapley(nn.Module):
    def __init__(self, output_size):
        super().__init__()
        self.output_size = output_size
        self.down1 = conv_block(4, 64)
        self.downsample1 = nn.MaxPool2d(2)
        self.down2 = conv_block(64, 128)
        self.downsample2 = nn.MaxPool2d(2)
        self.bottleneck = conv_block(128, 256)
        self.upsample1 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.up1 = conv_block(256, 128)
        self.upsample2 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.up2 = conv_block(128, 64)
        self.final = nn.Conv2d(64, 4 * output_size, 1)

    def forward(self, x):
        batch_size = x.shape[0]
        l1 = self.down1(x) 
        l2 = self.downsample1(l1)
        l3 = self.down2(l2) 
        l4 = self.downsample2(l3)
        l5 = self.bottleneck(l4) 
        l6 = self.upsample1(l5)
        l7 = self.up1(torch.cat((l3, l6), dim=1))
        l8 = self.upsample2(l7)
        l9 = self.up2(torch.cat((l1, l8), dim=1))
        l10 = self.final(l9)
        l11 = l10.view(batch_size, 4, self.output_size, 84, 84).permute(0, 1, 3, 4, 2)
        return l11
    
shap = Shapley(output_size=1)
import numpy as np
print(sum([np.prod(p.shape) for p in shap.parameters()]))

print(sum([np.prod(p.shape) for p in agent.parameters()]))


1863620
3370821


In [145]:
def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(),
    )

class ShapleyBig(nn.Module):
    def __init__(self, output_size=1):
        super().__init__()
        self.down1 = conv_block(4, 64)
        self.downsample1 = nn.MaxPool2d(2)
        self.down2 = conv_block(64, 128)
        self.downsample2 = nn.MaxPool2d(2)
        self.down3 = conv_block(128, 256)
        self.downsample3 = nn.MaxPool2d(3)
        self.bottleneck = conv_block(256, 512)
        self.upsample1 = nn.ConvTranspose2d(512, 256, 3, stride=3)
        self.up1 = conv_block(512, 256)
        self.upsample2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.up2 = conv_block(256, 128)
        self.upsample3 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.up3 = conv_block(128, 64)
        self.final = nn.Conv2d(64, 16, 1)

    def forward(self, x):
        batches = x.shape[0]
        l1 = self.down1(x) 
        l2 = self.downsample1(l1)
        l3 = self.down2(l2) 
        l4 = self.downsample2(l3)
        l5 = self.down3(l4)
        l6 = self.downsample3(l5)
        l7 = self.bottleneck(l6)
        l8 = self.upsample1(l7)
        l9 = self.up1(torch.cat((l5, l8), dim=1))
        l10 = self.upsample2(l9)
        l11 = self.up2(torch.cat((l3, l10), dim=1))
        l12 = self.upsample3(l11)
        l13 = self.up3(torch.cat((l1, l12), dim=1))
        l14 = self.final(l13)
        res = l14.view(-1, 4, 4, 84, 84).permute(0, 1, 3, 4, 2)
        return res
    
shap = ShapleyBig()
import numpy as np
print(sum([np.prod(p.shape) for p in shap.parameters()]))
shap(obs)

8354256


tensor([[[[[ 9.5206e-04, -7.0865e-02, -5.5784e-02,  1.2035e-01],
           [ 5.2162e-03, -7.0235e-02, -5.6360e-02,  1.2476e-01],
           [ 4.5120e-03, -7.0359e-02, -5.6502e-02,  1.2307e-01],
           ...,
           [ 4.7372e-03, -7.0257e-02, -5.6677e-02,  1.2386e-01],
           [ 3.3653e-03, -6.9572e-02, -5.5976e-02,  1.2215e-01],
           [ 6.9273e-03, -7.0389e-02, -5.4075e-02,  1.2433e-01]],

          [[-5.7801e-04, -7.1526e-02, -5.3381e-02,  1.1969e-01],
           [ 4.8212e-03, -6.9355e-02, -5.4752e-02,  1.2262e-01],
           [ 5.4842e-03, -6.9951e-02, -5.5879e-02,  1.2275e-01],
           ...,
           [ 5.2069e-03, -6.9288e-02, -5.5503e-02,  1.2182e-01],
           [ 4.9991e-03, -7.0978e-02, -5.6257e-02,  1.2161e-01],
           [ 7.8949e-03, -7.1487e-02, -5.4953e-02,  1.2308e-01]],

          [[-1.1835e-03, -7.1428e-02, -5.3649e-02,  1.1911e-01],
           [ 4.6702e-03, -7.0933e-02, -5.3937e-02,  1.2421e-01],
           [ 6.5456e-03, -7.1106e-02, -5.4407e-02,  1.

In [ ]:
print(obs.shape)
print(agent.to('cpu').get_value(obs).shape)
print(agent.to('cpu').get_action_logits(obs).shape)

torch.Size([8, 4, 84, 84])
torch.Size([8, 1])
torch.Size([8, 4])


In [125]:
shap = Shapley(output_size=4)


In [143]:
shap(obs).shape
mask = torch.rand_like(obs) < 0.5
# print(mask)
mask[:, :, :, :] = False
mask[0, 0, 0, 0] = True
mask[0, 0, 0, 1] = True
print(mask.shape)
res= shap(obs)
res2=(res * mask.unsqueeze(-1))
d = tuple(range(1, res2.dim() - 1))
print(res2.sum(dim=d))
print(res2.sum(dim=d).shape)
print(res2)


torch.Size([8, 4, 84, 84])
tensor([[ 0.2162, -0.2016, -0.1517,  0.1745],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000]], grad_fn=<SumBackward1>)
torch.Size([8, 4])
tensor([[[[[ 0.1081, -0.1006, -0.0747,  0.0875],
           [ 0.1081, -0.1010, -0.0770,  0.0870],
           [ 0.0000, -0.0000, -0.0000,  0.0000],
           ...,
           [ 0.0000, -0.0000, -0.0000,  0.0000],
           [ 0.0000, -0.0000, -0.0000,  0.0000],
           [ 0.0000, -0.0000, -0.0000,  0.0000]],

          [[ 0.0000, -0.0000, -0.0000,  0.0000],
           [ 0.0000, -0.0000, -0.0000,  0.0000],
           [ 0.0000, -0.0000, -0.0000,  0.0000],
           ...,
           [ 0.0000, -0.0000, -0.0000,  0.0000],
           [ 0.0000, -0.0000, -0.0000